In [5]:
import os
import ikomia 
from ikomia.utils import ik
from ikomia.dataprocess import workflow
import cv2
import numpy as np 

os.environ['IKOMIA_USER'] = "demo"
os.environ['IKOMIA_PWD'] = "jH4q72DApbRPa4k"

ikomia.authenticate()

### Setting up workflow

In [6]:
#  Workflow initialization
wf = workflow.create("Face detection and blur")

# Add kornia face detection algo to workflow
face_dect_id, face_dect = wf.add_task("infer_face_detection_kornia")

# Add open cv guaussian blur to the workflow
blurr_id, blurr = wf.add_task(ik.ocv_gaussian_blur)

# Set parameters
blurr_params = {
    ik.ocv_gaussian_blur_param.sigmaX: 45.0,
    ik.ocv_gaussian_blur_param.sigmaY: 45.0,
    ik.ocv_gaussian_blur_param.borderType: 2,
    ik.ocv_gaussian_blur_param.sizeX: 29,
    ik.ocv_gaussian_blur_param.sizeY: 29,
}

wf.set_parameters(task_id=blurr_id, params=blurr_params)

# Connect tasks
wf.connect_tasks(wf.getRootID(), face_dect_id)
wf.connect_tasks(face_dect_id, blurr_id)



### Detection and blur from image


In [7]:
# Inference
img_path = "images/detect_image/img1.png"
wf.run_on(img_path)

img = wf.get_image(blurr_id, index=0)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imshow('object detection (press \'q\' to quit)', img)

cv2.waitKey(-1)

Will run on cuda


c:\Users\poopo\Desktop\Ikomia\SegFormer\ikomia-seg\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

### Detection and blur from webcam


In [8]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    wf.run_on(image_np)
    
    img = wf.get_image(blurr_id, index=0)

    cv2.imshow('object detection (press \'q\' to quit)', img)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()